# Step for Analysis

In [10]:
# Import packages and files
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind, pearsonr
import statsmodels.api as sm
import statsmodels.formula.api as smf


demo_df = pd.read_sas('/content/DEMO_L (1).xpt', format = 'xport')
bpo_df = pd.read_sas('/content/BPXO_L.xpt', format = 'xport')
kidney_df = pd.read_sas('/content/KIQ_U_L.xpt', format = 'xport')
hepb_df = pd.read_sas('/content/HEPB_S_L.xpt', format = 'xport')
phys_df = pd.read_sas('/content/PAQ_L.xpt', format = 'xport')
vitd_df = pd.read_sas('/content/VID_L.xpt', format = 'xport')
weigh_f = pd.read_sas('/content/WHQ_L.xpt', format = 'xport')


In [2]:
hepb_df

,SEQN,WTPH2YR,LBXHBS
0,130378.0,5.604213e+04,2.0
1,130379.0,3.743571e+04,2.0
2,130380.0,8.532884e+04,2.0
3,130381.0,5.397605e-79,NaN
4,130382.0,5.963893e+04,2.0
...,...,...,...
8606,142306.0,5.397605e-79,NaN
8607,142307.0,6.899418e+04,2.0
8608,142308.0,5.397605e-79,NaN
8609,142309.0,4.628442e+04,1.0


In [3]:
# Merge of files

###Notes sequences are "Respondent Sequence Number" which is an unique identifer
demo = demo_df[["SEQN", "DMDMARTZ", "DMDEDUC2", "RIDAGEYR"]].copy()
bpx  = bpo_df[["SEQN", "BPXOSY3", "BPXODI3"]].copy()
vitd = vitd_df[["SEQN", "LBDVD2LC"]].copy()
hepb = hepb_df[["SEQN", "LBXHBS"]].copy()
kid  = kidney_df[["SEQN", "KIQ022"]].copy()
paq  = phys_df[["SEQN", "PAD680"]].copy()
whq  = weigh_f[["SEQN", "WHD020"]].copy()

df = demo.merge(bpx, on="SEQN", how="left") \
         .merge(vitd, on="SEQN", how="left") \
         .merge(hepb, on="SEQN", how="left") \
         .merge(kid, on="SEQN", how="left") \
         .merge(paq, on="SEQN", how="left") \
         .merge(whq, on="SEQN", how="left")

df

,SEQN,DMDMARTZ,DMDEDUC2,RIDAGEYR,BPXOSY3,BPXODI3,LBDVD2LC,LBXHBS,KIQ022,PAD680,WHD020
0,130378.0,1.0,5.0,43.0,132.0,94.0,1.0,2.0,2.0,360.0,190.0
1,130379.0,1.0,5.0,66.0,113.0,76.0,1.0,2.0,2.0,480.0,220.0
2,130380.0,1.0,3.0,44.0,104.0,76.0,1.0,2.0,2.0,240.0,150.0
3,130381.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130382.0,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11928,142306.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11929,142307.0,3.0,5.0,49.0,131.0,72.0,1.0,2.0,2.0,480.0,206.0
11930,142308.0,1.0,4.0,50.0,112.0,74.0,NaN,NaN,2.0,600.0,174.0
11931,142309.0,2.0,4.0,40.0,128.0,81.0,1.0,1.0,2.0,240.0,200.0


In [6]:
df.dtypes
# Remove 7777 and 9999
df.loc[df["PAD680"].isin([7777, 9999]), "PAD680"] = pd.NA
df.loc[df["WHD020"].isin([7777, 9999]), "WHD020"] = pd.NA

# Marriage
df.loc[df["DMDMARTZ"] == "1", "DMDMARTZ"] = "Married"
df.loc[df["DMDMARTZ"] != "Married", "DMDMARTZ"] = "Not married"

# Education
df.loc[df["DMDEDUC2"] == "5", "DMDEDUC2"] = "Bachelor+"
df.loc[df["DMDEDUC2"] != "Bachelor+", "DMDEDUC2"] = "< Bachelor"

# Vitamin D 2 Level LOD-Level of Detection
df.loc[df["LBDVD2LC"] == "1", "LBDVD2LC"] = "Below LOD"
df.loc[df["LBDVD2LC"] == "0", "LBDVD2LC"] = "At/Above LOD"
df.loc[~df["LBDVD2LC"].isin(["Below LOD", "At/Above LOD"]), "LBDVD2LC"] = pd.NA

# HEP B Levels
df.loc[df["LBXHBS"] == "1", "LBXHBS"] = "Positive"
df.loc[df["LBXHBS"] == "2", "LBXHBS"] = "Negative"
df.loc[~df["LBXHBS"].isin(["Positive", "Negative"]), "LBXHBS"] = pd.NA

# Kidney Ranges- Ever told you had weak/failing kidneys?
df.loc[df["KIQ022"] == "1", "KIQ022"] = "Yes"
df.loc[df["KIQ022"] == "2", "KIQ022"] = "No"
df.loc[~df["KIQ022"].isin(["Yes", "No"]), "KIQ022"] = pd.NA

df

,SEQN,DMDMARTZ,DMDEDUC2,RIDAGEYR,BPXOSY3,BPXODI3,LBDVD2LC,LBXHBS,KIQ022,PAD680,WHD020
0,130378.0,Married,Bachelor+,43.0,132.0,94.0,Below LOD,Negative,No,360.0,190.0
1,130379.0,Married,Bachelor+,66.0,113.0,76.0,Below LOD,Negative,No,480.0,220.0
2,130380.0,Married,< Bachelor,44.0,104.0,76.0,Below LOD,Negative,No,240.0,150.0
3,130381.0,Not married,< Bachelor,5.0,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN
4,130382.0,Not married,< Bachelor,2.0,NaN,NaN,<NA>,Negative,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11928,142306.0,Not married,< Bachelor,9.0,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN
11929,142307.0,Not married,Bachelor+,49.0,131.0,72.0,Below LOD,Negative,No,480.0,206.0
11930,142308.0,Married,< Bachelor,50.0,112.0,74.0,<NA>,<NA>,No,600.0,174.0
11931,142309.0,Not married,< Bachelor,40.0,128.0,81.0,Below LOD,Positive,No,240.0,200.0


In [11]:
# QUESTION 1
## Make a cross-tab table
table = pd.crosstab(df["DMDMARTZ"], df["DMDEDUC2"])
print("Contingency Table:\n", table, "\n")

## Run chi-square test
chi2, p, dof, expected = chi2_contingency(table)
print(f"Chi-square = {chi2:.3f}")
print(f"Degrees of freedom = {dof}")
print(f"P-value = {p:.4f}")

Contingency Table:
 DMDEDUC2     < Bachelor  Bachelor+
DMDMARTZ                          
Married            2505       1631
Not married        6803        994 

Chi-square = 1120.026
Degrees of freedom = 1
P-value = 0.0000


In [12]:
# Question 2

## T-test
data = df.dropna(subset=["PAD680","DMDMARTZ"])
married = data.loc[data["DMDMARTZ"]=="Married","PAD680"]
not_married = data.loc[data["DMDMARTZ"]=="Not married","PAD680"]

t, p = ttest_ind(married, not_married, equal_var=False)
print(f"T = {t:.3f}, p = {p:.4f}")

T = -3.870, p = 0.0001


In [18]:
# Question 3

## Regression
data3 = df.dropna(subset=["BPXOSY3","RIDAGEYR","DMDMARTZ"]).copy()
data3["married_num"] = data3["DMDMARTZ"].map({"Married":1,"Not married":0})

model = smf.ols("BPXOSY3 ~ RIDAGEYR + married_num", data=data3).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                BPXOSY3   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     1312.
Date:                Sat, 01 Nov 2025   Prob (F-statistic):               0.00
Time:                        01:37:36   Log-Likelihood:                -31311.
No. Observations:                7480   AIC:                         6.263e+04
Df Residuals:                    7477   BIC:                         6.265e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     100.2281      0.410    244.241      

In [17]:
# Question 4

## Non-Paramatic Correlation
data4 = df.dropna(subset=["WHD020","PAD680"])
r, p = pearsonr(data4["WHD020"], data4["PAD680"])
print(f"Correlation r = {r:.3f}, p = {p:.4f}")



Correlation r = 0.156, p = 0.0000


In [19]:
# Question 5

## Distolic blood pressure and weak/kindney failure T-test
data5 = df.dropna(subset=["BPXODI3","KIQ022"])
yes = data5.loc[data5["KIQ022"]=="Yes","BPXODI3"]
no  = data5.loc[data5["KIQ022"]=="No","BPXODI3"]

t, p = ttest_ind(yes, no, equal_var=False)
print(f"T = {t:.3f}, p = {p:.4f}")


T = -4.234, p = 0.0000
